# Texas Tow Trucks

We're going to scrape some [tow trucks in Texas](https://www.tdlr.texas.gov/tools_search/).

## Import your imports

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

## Search for the TLDR Number `006179570C`, and scrape the information on that company

Using [license information system](https://www.tdlr.texas.gov/tools_search/), find information about the tow truck number above, displaying the

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

If you can't figure a 'nice' way to locate something, your two last options might be:

- **Find a "parent" element, then dig inside**
- **Find all of a type of element** (like we did with `td` before) and get the `[0]`, `[1]`, `[2]`, etc
- **XPath** (inspect an element, Copy > Copy XPath)

These kinds of techniques tend to break when you're on other result pages, but... maybe not! You won't know until you try.

> - *TIP: When you use xpath, you CANNOT use double quotes or Python will get confused. Use single quotes.*
> - *TIP: You can clean your data up if you want to, or leave it dirty to clean later*
> - *TIP: The address part can be tough, but you have a few options. You can use a combination of `.split` and list slicing to clean it now, or clean it later in the dataframe with regular expressions. Or other options, too, probably*

In [154]:
driver = webdriver.Chrome()
url = "https://www.tdlr.texas.gov/tools_search/"
driver.get(url)

In [161]:
url = "https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber={trdl_n}".format(trdl_n = "006448786C")
driver.get(url)

# - The business name
biz_name_d = driver.find_elements_by_tag_name("tr")[4].find_elements_by_tag_name("td")[0]
#clean it up
biz_name = biz_name_d.text.replace("Name:   ", "")
print("business name:", biz_name)

# - Owner/operator
if len(driver.find_element_by_id("t1").find_elements_by_tag_name("tbody")[-2].find_elements_by_tag_name("td")) == 10:
    owner_d1 = driver.find_elements_by_tag_name("tr")[5].find_elements_by_tag_name("td")[0]
    owner1 = owner_d1.text.replace("Owner/Officer:   ", "")
    
    owner_d2 = driver.find_elements_by_tag_name("tr")[6].find_elements_by_tag_name("td")[0]
    owner2 = owner_d2.text.replace("Owner/Officer:   ", "")
    print("owner/operator1:", owner1)
    print("owner/operator2:", owner2)
else:
    owner_d1 = driver.find_elements_by_tag_name("tr")[5].find_elements_by_tag_name("td")[0]
    owner1 = owner_d1.text.replace("Owner/Officer:   ", "")
    owner2 = "N/A"
    print("owner/operator1:", owner1)
    print("owner/operator2:", owner2)   

# - Phone number
phone_d = driver.find_element_by_id("t1").find_elements_by_tag_name("tbody")[-2].find_elements_by_tag_name("td")[-2]
phone = phone_d.text.replace("Phone:   ", "")
print("phone number:", phone)

# - License status (Active, Expired, Etc)
status_d = driver.find_element_by_id("t1").find_elements_by_tag_name("tbody")[4].find_elements_by_tag_name("td")[1]
status = status_d.text.replace("Status:  ", "")
print("license status:", status)

# - Physical address
address = driver.find_element_by_id("t1").find_elements_by_tag_name("tbody")[-1]
p_address_d = address.text.split('\n')[-2:]
p_address = ", ".join(p_address_d)
print("physical address:", p_address)



business name: HYSMITH AUTOMOTIVE
owner/operator1: WILLIAM THOMAS HYSMITH / PRESIDENT
owner/operator2: ASHLEY ERIN HYSMITH / TREASURER
phone number: 940-521-0294
license status: Active
physical address: 1210 US 380 BYPASS, GRAHAM, TX. 76450


# Adapt this to work inside of a single cell

Double-check that it works. You want it to print out all of the details.

In [124]:
#Get URL
url = "https://www.tdlr.texas.gov/tools_search/"
driver.get(url)

#Search
tldr_n = driver.find_element_by_id("mcrdata")
tldr_n.send_keys("006179570C")

search_button = driver.find_element_by_id("submit3")
search_button.click()

##Get info
# - The business name
biz_name_d = driver.find_elements_by_tag_name("tr")[4].find_elements_by_tag_name("td")[0]
#clean it up
biz_name = biz_name_d.text.replace("Name:   ", "")
print("business name:", biz_name)

# - Owner/operator
owner_d = driver.find_elements_by_tag_name("tr")[5].find_elements_by_tag_name("td")[0]
owner = owner_d.text.replace("Owner/Officer:   ", "")
print("owner/operator:", owner)

# - Phone number
phone_d = driver.find_elements_by_tag_name("tr")[6].find_elements_by_tag_name("td")[0]
phone = phone_d.text.replace("Phone:   ", "")
print("phone number:", phone)

# - License status (Active, Expired, Etc)
status_d = driver.find_elements_by_tag_name("tr")[7].find_elements_by_tag_name("td")[1]
status = status_d.text.replace("Status:  ", "")
print("license status:", status)

# - Physical address
address = driver.find_elements_by_tag_name("tr")[8].find_elements_by_tag_name("td")[1]
p_address_d = address.text.split('\n')[-2:]
p_address = ", ".join(p_address_d)
print("physical address:", p_address)

business name: B.D. SMITH TOWING
owner/operator: BRANDT SMITH / OWNER
phone number: 8173330706
license status: Expired
physical address: 13619 BRETT JACKSON RD., FORT WORTH, TX. 76179


# Using .apply to find data about SEVERAL tow truck companies

The file `trucks-subset.csv` has information about the trucks, we'll use it to find the pages to scrape.

### Open up `trucks-subset.csv` and save it into a dataframe

In [61]:
import pandas as pd

In [64]:
df = pd.read_csv("trucks-subset.csv")
df.head()
df.shape
df.dtypes

TDLR Number    object
dtype: object

## Go through each row of the dataset, displaying the URL you will need to scrape for the information on that row

You don't have to actually use the search form for each of these - look at the URL you're on, it has the number in it!

For example, one URL might look like `https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C`.

- *TIP: Use .apply and a function*
- *TIP: Unlike the Yelp example, you'll need to build this URL from pieces*
- *TIP: You probably don't want to `print` unless you're going to fix it for the next question 
- *TIP: pandas won't showing you the entire url! Run `pd.set_option('display.max_colwidth', -1)` to display aaaalll of the text in a cell*

In [169]:
def get_truck_info(row):
    url = "https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber={trdl_n}".format(trdl_n = row["TDLR Number"])
    driver.get(url)

    # - The business name
    biz_name_d = driver.find_elements_by_tag_name("tr")[4].find_elements_by_tag_name("td")[0]
    #clean it up
    biz_name = biz_name_d.text.replace("Name:   ", "")

    # - Owner/operator
    if len(driver.find_element_by_id("t1").find_elements_by_tag_name("tbody")[-2].find_elements_by_tag_name("td")) == 10:
        owner_d1 = driver.find_elements_by_tag_name("tr")[5].find_elements_by_tag_name("td")[0]
        owner1 = owner_d1.text.replace("Owner/Officer:   ", "")
    
        owner_d2 = driver.find_elements_by_tag_name("tr")[6].find_elements_by_tag_name("td")[0]
        owner2 = owner_d2.text.replace("Owner/Officer:   ", "")
    else:
        owner_d1 = driver.find_elements_by_tag_name("tr")[5].find_elements_by_tag_name("td")[0]
        owner1 = owner_d1.text.replace("Owner/Officer:   ", "")
        owner2 = "N/A"

    # - Phone number
    phone_d = driver.find_element_by_id("t1").find_elements_by_tag_name("tbody")[-2].find_elements_by_tag_name("td")[-2]
    phone = phone_d.text.replace("Phone:   ", "")

    # - License status (Active, Expired, Etc)
    status_d = driver.find_element_by_id("t1").find_elements_by_tag_name("tbody")[4].find_elements_by_tag_name("td")[1]
    status = status_d.text.replace("Status:  ", "")

    # - Physical address
    address = driver.find_element_by_id("t1").find_elements_by_tag_name("tbody")[-1]
    p_address_d = address.text.split('\n')[-2:]
    p_address = ", ".join(p_address_d)
    
    return pd.Series({
        "business name" : biz_name,
        "owner/operator1" : owner1,
        "owner/operator2" : owner2,
        "phone number" : phone,
        "license status" : status,
        "physical address" : p_address,
        "url" : url,
        "TDLR number" : row["TDLR Number"]
    })


In [163]:
df_with_info = df.apply(get_truck_info, axis = 1)
df_with_info

,business name,owner/operator1,owner/operator2,phone number,license status,physical address,url,TDLR number
0,AUGUSTUS E SMITH,AUGUSTUS EUGENE SMITH / OWNER,N/A,9032276464,Active,"103 N MAIN ST, BONHAM, TX. 75418",https://www.tdlr.texas.gov/tools_search/mccs_d...,006507931C
1,B.D. SMITH TOWING,BRANDT SMITH / OWNER,N/A,8173330706,Expired,"13619 BRETT JACKSON RD., FORT WORTH, TX. 76179",https://www.tdlr.texas.gov/tools_search/mccs_d...,006179570C
2,BARRY MICHAEL SMITH,BARRY MICHAEL SMITH / OWNER,N/A,8066544404,Active,"4501 W CEMETERY RD, CANYON, TX. 79015",https://www.tdlr.texas.gov/tools_search/mccs_d...,006502097C


### Save this URL into a new column of your dataframe, called `url`

- *TIP: Use a function and `.apply`*
- *TIP: Be sure to use `return`*

In [130]:
#Oh ... I sort of did all this

## Go through each row of the dataset, printing out information about each tow truck company.

Now will be **scraping** inside of your function.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

Just print it out for now.

- *TIP: use .apply*
- *TIP: You'll be using the code you wrote before, but converted into a function*
- *TIP: Remember how the TDLR Number is in the URL? You don't need to do the form submission if you don't want!*
- *TIP: Make sure you adjust any variables so you don't scrape the same page again and again*

In [131]:
#and this

## Scrape the following information for each row of the dataset, and save it into new columns in your dataframe.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

It's basically what we did before, but using the function a little differently.

- *TIP: Same as above, but you'll be returning a `pd.Series` and the `.apply` line is going to be a lot longer*
- *TIP: Save it to a new dataframe!*
- *TIP: Make sure you change your `df` variable names correctly if you're cutting and pasting - there are a few so it can get tricky*

In [132]:
#and this

### Save your dataframe as a CSV named `tow-trucks-extended.csv`

In [148]:
df_with_info.to_csv("tow-trucks-extended.csv", index = False)

### Re-open your dataframe to confirm you didn't save any extra weird columns

In [149]:
df_test = pd.read_csv("tow-trucks-extended.csv")
df_test

,business name,owner/operator1,owner/operator2,phone number,license status,physical address,url,TDLR number
0,AUGUSTUS E SMITH,AUGUSTUS EUGENE SMITH / OWNER,NaN,9032276464,Active,"103 N MAIN ST, BONHAM, TX. 75418",https://www.tdlr.texas.gov/tools_search/mccs_d...,006507931C
1,B.D. SMITH TOWING,BRANDT SMITH / OWNER,NaN,8173330706,Expired,"13619 BRETT JACKSON RD., FORT WORTH, TX. 76179",https://www.tdlr.texas.gov/tools_search/mccs_d...,006179570C
2,BARRY MICHAEL SMITH,BARRY MICHAEL SMITH / OWNER,NaN,8066544404,Active,"4501 W CEMETERY RD, CANYON, TX. 79015",https://www.tdlr.texas.gov/tools_search/mccs_d...,006502097C


## Process the entire `tow-trucks.csv` file

We just did it on a short subset so far. Now try it on all of the tow trucks. **Save as the same filename as before**

In [167]:
df_all = pd.read_csv("tow-trucks.csv")
df_all.shape

(20, 1)

In [170]:
df_all_with_info = df_all.apply(get_truck_info, axis = 1)
df_all_with_info

,business name,owner/operator1,owner/operator2,phone number,license status,physical address,url,TDLR number
0,AUGUSTUS E SMITH,AUGUSTUS EUGENE SMITH / OWNER,N/A,9032276464,Active,"103 N MAIN ST, BONHAM, TX. 75418",https://www.tdlr.texas.gov/tools_search/mccs_d...,006507931C
1,B.D. SMITH TOWING,BRANDT SMITH / OWNER,N/A,8173330706,Expired,"13619 BRETT JACKSON RD., FORT WORTH, TX. 76179",https://www.tdlr.texas.gov/tools_search/mccs_d...,006179570C
2,BARRY MICHAEL SMITH,BARRY MICHAEL SMITH / OWNER,N/A,8066544404,Active,"4501 W CEMETERY RD, CANYON, TX. 79015",https://www.tdlr.texas.gov/tools_search/mccs_d...,006502097C
3,HEATH SMITH,HEATH A SMITH / OWNER,N/A,940-552-0687,Expired,"1529 WILBARGER ST, VERNON, TX. 76384",https://www.tdlr.texas.gov/tools_search/mccs_d...,006494912C
4,HEATH SMITH,HEATH A SMITH / OWNER,N/A,9405520687,Expired,"1529 WILBARGER ST, VERNON, TX. 76384",https://www.tdlr.texas.gov/tools_search/mccs_d...,0649468VSF
5,HYSMITH AUTOMOTIVE,WILLIAM THOMAS HYSMITH / PRESIDENT,ASHLEY ERIN HYSMITH / TREASURER,940-521-0294,Active,"1210 US 380 BYPASS, GRAHAM, TX. 76450",https://www.tdlr.texas.gov/tools_search/mccs_d...,006448786C
6,HYSMITH AUTOMOTIVE & TRUCK REPAIR INC,HYSMITH ERIN ASHLEY / TREASURER,WILLIAM THOMAS HYSMITH / PRESIDENT,9405210294,Suspended,"927 LOVING HWY, GRAHAM, TX. 76450",https://www.tdlr.texas.gov/tools_search/mccs_d...,0648444VSF
7,HYSMITH AUTOMOTIVE & TRUCK REPAIR INC,WILLIAM THOMAS HYSMITH / PRESIDENT,ASHLEY ERIN HYSMITH / TREASURER,9405210294,Active,"1210 380 BYPASS, GRAHAM, TX. 76450",https://www.tdlr.texas.gov/tools_search/mccs_d...,0651667VSF
8,JEFF & WENDY SMITH,JEFF SMITH / PARTNER,WENDY SMITH / PARTNER,903 586 6234,Suspended,"10842 FM 2138 N, JACKSONVILLE, TX. 75766",https://www.tdlr.texas.gov/tools_search/mccs_d...,006017767C
9,JEFF SMITH,JEFFREY JOHN SMITH / OWNER,N/A,8324354670,Active,"4338 HARVEY RD, CROSBY, TX. 77532",https://www.tdlr.texas.gov/tools_search/mccs_d...,006495492C


In [171]:
df_all_with_info.to_csv("tow-trucks-extended.csv", index = False)

In [121]:
df_test = pd.read_csv("tow-trucks-extended.csv")
df_test

,business name,owner/operator,phone number,license status,physical address,url,TDLR number
0,AUGUSTUS E SMITH,AUGUSTUS EUGENE SMITH / OWNER,9032276464,Active,"103 N MAIN ST, BONHAM, TX. 75418",https://www.tdlr.texas.gov/tools_search/mccs_d...,006507931C
1,B.D. SMITH TOWING,BRANDT SMITH / OWNER,8173330706,Expired,"13619 BRETT JACKSON RD., FORT WORTH, TX. 76179",https://www.tdlr.texas.gov/tools_search/mccs_d...,006179570C
2,BARRY MICHAEL SMITH,BARRY MICHAEL SMITH / OWNER,8066544404,Active,"4501 W CEMETERY RD, CANYON, TX. 79015",https://www.tdlr.texas.gov/tools_search/mccs_d...,006502097C
3,HEATH SMITH,HEATH A SMITH / OWNER,940-552-0687,Expired,"1529 WILBARGER ST, VERNON, TX. 76384",https://www.tdlr.texas.gov/tools_search/mccs_d...,006494912C
4,HEATH SMITH,HEATH A SMITH / OWNER,9405520687,Expired,"1529 WILBARGER ST, VERNON, TX. 76384",https://www.tdlr.texas.gov/tools_search/mccs_d...,0649468VSF
5,HYSMITH AUTOMOTIVE,WILLIAM THOMAS HYSMITH / PRESIDENT,Owner/Officer: ASHLEY ERIN HYSMITH / TREASURER,Active,"1210 US 380 BYPASS, GRAHAM, TX. 76450",https://www.tdlr.texas.gov/tools_search/mccs_d...,006448786C
6,HYSMITH AUTOMOTIVE & TRUCK REPAIR INC,HYSMITH ERIN ASHLEY / TREASURER,Owner/Officer: WILLIAM THOMAS HYSMITH / PRES...,Suspended,"927 LOVING HWY, GRAHAM, TX. 76450",https://www.tdlr.texas.gov/tools_search/mccs_d...,0648444VSF
7,HYSMITH AUTOMOTIVE & TRUCK REPAIR INC,WILLIAM THOMAS HYSMITH / PRESIDENT,Owner/Officer: ASHLEY ERIN HYSMITH / TREASURER,Active,"1210 380 BYPASS, GRAHAM, TX. 76450",https://www.tdlr.texas.gov/tools_search/mccs_d...,0651667VSF
8,JEFF & WENDY SMITH,JEFF SMITH / PARTNER,Owner/Officer: WENDY SMITH / PARTNER,Suspended,"10842 FM 2138 N, JACKSONVILLE, TX. 75766",https://www.tdlr.texas.gov/tools_search/mccs_d...,006017767C
9,JEFF SMITH,JEFFREY JOHN SMITH / OWNER,8324354670,Active,"4338 HARVEY RD, CROSBY, TX. 77532",https://www.tdlr.texas.gov/tools_search/mccs_d...,006495492C
